# import libraries

In [6]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import plotly.express as px
import random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, metrics
from collections import Counter
import re
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px
import plotly.graph_objects as go


# loading data

In [7]:
df = pd.read_csv("Data/2007_22_keywords.csv", delimiter='\t')
df


,pmid,year,month,title,title_cleaned,lemma,lemma_without_stopwords,tfidf_keywords
0,17690230,2007,Aug,"The visual pigments of a deep-sea teleost, the...",the visual pigments of a deep sea teleost the...,the visual pigment of a deep sea teleost the...,visual pigment deep sea teleost pearl eye scop...,teleost pearl pigment sea deep
1,17682436,2007,Jul,[Primary liver carcinoid tumor complicated wit...,primary liver carcinoid tumor complicated wit...,primary liver carcinoid tumor complicate wit...,liver carcinoid tumor complicate colon cancer,carcinoid complicate colon liver tumor
2,17875888,2007,Sep,Diffuse nodules in a woman with renal failure:...,diffuse nodules in a woman with renal failure ...,diffuse nodule in a woman with renal failure ...,diffuse nodule woman renal failure chronic top...,gout nodule diffuse failure renal
3,17628980,2007,Apr,[Feature of ACh-sensitive potassium current in...,feature of ach sensitive potassium current in...,feature of ach sensitive potassium current i...,feature ach sensitive potassium current outer ...,cochlea outer guinea hair potassium
4,17666563,2007,Aug,Helicobacter pylori eradication in the managem...,helicobacter pylori eradication in the managem...,helicobacter pylori eradication in the managem...,helicobacter pylori eradication idiopathic pul...,eradication helicobacter pylori idiopathic fib...
...,...,...,...,...,...,...,...,...
1638182,36452147,2022,NaN,Meta-analysis of substitution value of maize w...,meta analysis of substitution value of maize w...,meta analysis of substitution value of maize w...,substitution value maize cassava manihot escul...,cassava substitution broiler maize chicken
1638183,35583455,2022,Aug,Intraplaque calcium and its relation with the ...,intraplaque calcium and its relation with the ...,intraplaque calcium and its relation with the ...,intraplaque calcium relation progression carot...,carotid relation calcium progression
1638184,35565964,2022,Apr,Mechanistic Insights into the Ameliorating Eff...,mechanistic insights into the ameliorating eff...,mechanistic insight into the ameliorate effect...,mechanistic insight ameliorate melanogenesis p...,mechanistic ameliorate melanoma derivative ins...
1638185,35080294,2022,01,Controversies in the treatment of mild asthma....,controversies in the treatment of mild asthma ...,controversy in the treatment of mild asthma ...,controversy mild asthma novelty practical impl...,novelty controversy practical mild asthma


In [8]:
df = df.dropna()

# Counter

Pre všetky články

In [38]:
text_tfidf = re.findall('\w+',' '.join(df['tfidf_keywords'].tolist()))
#len(text_tfidf)

In [40]:
counts =  Counter(text_tfidf)
counts.most_common()[:20] 

[('cell', 15100),
 ('cancer', 13895),
 ('care', 8071),
 ('child', 6835),
 ('protein', 6458),
 ('therapy', 6456),
 ('base', 6242),
 ('gene', 5949),
 ('drug', 5332),
 ('breast', 5205),
 ('19', 5024),
 ('disorder', 5015),
 ('pain', 4995),
 ('surgery', 4979),
 ('infection', 4814),
 ('associate', 4592),
 ('outcome', 4591),
 ('literature', 4569),
 ('lung', 4453),
 ('tumor', 4428)]

## Podľa rokov

### tf-idf

In [12]:
yearly_tfidf = {}
for y in df['year'].unique():
    yearly_tfidf[y] = re.findall('\w+',' '.join(df[df['year']==y]['tfidf_keywords'].tolist()))

counts_tf_idf = {}
for y in df['year'].unique():
    counts_tf_idf[y] = Counter(yearly_tfidf[y])


In [13]:
for y in df['year'].unique():
    print(y)
    print(counts_tf_idf[y].most_common()[:10])
    print()

2007
[('cell', 776), ('cancer', 604), ('protein', 439), ('gene', 408), ('child', 381), ('therapy', 374), ('care', 341), ('drug', 285), ('receptor', 282), ('breast', 254)]

2008
[('cell', 785), ('cancer', 611), ('gene', 439), ('protein', 421), ('care', 396), ('child', 383), ('therapy', 329), ('drug', 296), ('receptor', 274), ('disorder', 272)]

2009
[('cell', 793), ('cancer', 649), ('care', 422), ('protein', 406), ('gene', 392), ('therapy', 353), ('child', 343), ('base', 294), ('drug', 284), ('receptor', 269)]

2010
[('cell', 902), ('cancer', 730), ('protein', 455), ('child', 453), ('care', 410), ('gene', 390), ('therapy', 379), ('base', 293), ('genetic', 290), ('surgery', 285)]

2011
[('cell', 884), ('cancer', 766), ('care', 430), ('therapy', 415), ('protein', 412), ('child', 392), ('gene', 369), ('base', 341), ('breast', 315), ('drug', 311)]

2012
[('cell', 892), ('cancer', 845), ('care', 441), ('protein', 416), ('child', 411), ('therapy', 371), ('gene', 354), ('base', 333), ('breast'

### Lemmatization

In [14]:
yearly_lemma = {}
for y in df['year'].unique():
    yearly_lemma[y] = re.findall('\w+',' '.join(df[df['year']==y]['lemma_without_stopwords'].tolist()))

counts_lemma_counts = {}
for y in df['year'].unique():
    counts_lemma_counts[y] = Counter(yearly_lemma[y])

for y in df['year'].unique():
    print(counts_lemma_counts[y].most_common()[:10])
    print()

[('cell', 4678), ('cancer', 2295), ('protein', 2231), ('1', 2098), ('2', 1995), ('gene', 1891), ('induce', 1888), ('s', 1601), ('receptor', 1544), ('base', 1467)]

[('cell', 4920), ('1', 2387), ('cancer', 2362), ('2', 2302), ('protein', 2233), ('induce', 1999), ('gene', 1900), ('s', 1733), ('base', 1629), ('receptor', 1515)]

[('cell', 5002), ('cancer', 2638), ('2', 2548), ('1', 2457), ('protein', 2220), ('induce', 1963), ('gene', 1921), ('base', 1842), ('s', 1830), ('3', 1502)]

[('cell', 5668), ('cancer', 2965), ('2', 2828), ('1', 2727), ('protein', 2415), ('induce', 2093), ('s', 2039), ('base', 2030), ('gene', 1945), ('receptor', 1617)]

[('cell', 5694), ('cancer', 3216), ('1', 2915), ('2', 2640), ('protein', 2321), ('base', 2253), ('induce', 2249), ('s', 1975), ('gene', 1958), ('3', 1608)]

[('cell', 5903), ('cancer', 3529), ('2', 2877), ('1', 2846), ('base', 2449), ('induce', 2381), ('protein', 2264), ('s', 2059), ('gene', 1905), ('associate', 1778)]

[('cell', 6122), ('cancer', 3

In [16]:
def perc_zast_slovo(slovo, tfidf=True, make_dict=False, print_out=True):

    if tfidf:
        counts = counts_tf_idf
    else:
        counts = counts_lemma_counts
    out = {}
    for y in df['year'].unique():
        pocet_cl = len(df[df['year']==y])
        s = round((counts[y][slovo]/pocet_cl)*100, 2)
        out[y]=s

        if print_out:
            print('rok: ',y)
            print('pocet clankov v roku: ', pocet_cl)
            print('percentualne zastupenie slova', slovo, s)
            print()
    if make_dict:
        return out

In [41]:
perc_zast_slovo('cancer', tfidf=False)

rok:  2007
pocet clankov v roku:  59224
percentualne zastupenie slova cancer 3.88

rok:  2008
pocet clankov v roku:  63128
percentualne zastupenie slova cancer 3.74

rok:  2009
pocet clankov v roku:  66038
percentualne zastupenie slova cancer 3.99

rok:  2010
pocet clankov v roku:  71049
percentualne zastupenie slova cancer 4.17

rok:  2011
pocet clankov v roku:  73521
percentualne zastupenie slova cancer 4.37

rok:  2012
pocet clankov v roku:  76466
percentualne zastupenie slova cancer 4.62

rok:  2013
pocet clankov v roku:  80048
percentualne zastupenie slova cancer 4.92

rok:  2014
pocet clankov v roku:  83281
percentualne zastupenie slova cancer 5.05

rok:  2015
pocet clankov v roku:  85801
percentualne zastupenie slova cancer 5.07

rok:  2016
pocet clankov v roku:  91782
percentualne zastupenie slova cancer 5.28

rok:  2017
pocet clankov v roku:  94741
percentualne zastupenie slova cancer 5.45

rok:  2018
pocet clankov v roku:  98186
percentualne zastupenie slova cancer 5.46

rok:

In [42]:
perc_zast_slovo('cancer', tfidf=True)

rok:  2007
pocet clankov v roku:  59224
percentualne zastupenie slova cancer 1.02

rok:  2008
pocet clankov v roku:  63128
percentualne zastupenie slova cancer 0.97

rok:  2009
pocet clankov v roku:  66038
percentualne zastupenie slova cancer 0.98

rok:  2010
pocet clankov v roku:  71049
percentualne zastupenie slova cancer 1.03

rok:  2011
pocet clankov v roku:  73521
percentualne zastupenie slova cancer 1.04

rok:  2012
pocet clankov v roku:  76466
percentualne zastupenie slova cancer 1.11

rok:  2013
pocet clankov v roku:  80048
percentualne zastupenie slova cancer 1.04

rok:  2014
pocet clankov v roku:  83281
percentualne zastupenie slova cancer 1.07

rok:  2015
pocet clankov v roku:  85801
percentualne zastupenie slova cancer 1.01

rok:  2016
pocet clankov v roku:  91782
percentualne zastupenie slova cancer 1.04

rok:  2017
pocet clankov v roku:  94741
percentualne zastupenie slova cancer 1.02

rok:  2018
pocet clankov v roku:  98186
percentualne zastupenie slova cancer 0.97

rok:

### make dataframe of counts for list of words

In [43]:
def make_df_counts(words, tfidf=False):
    word_count = {}
    for word in words:
        word_count[word] = perc_zast_slovo(word, tfidf=tfidf, print_out=False, make_dict=True)
    df_counts = pd.DataFrame.from_dict(data=word_count, orient='index')
    return df_counts

In [45]:
df_model = make_df_counts(['antibiotic', 'cell', 'covid', 'hiv', 'mouse'])
df_model = df_model.reset_index()

In [46]:
df_model

,index,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,antibiotic,0.26,0.22,0.24,0.25,0.25,0.27,0.26,0.30,0.30,0.33,0.36,0.38,0.44,0.40,0.40,0.45
1,cell,7.90,7.79,7.57,7.98,7.74,7.72,7.65,7.62,7.62,7.68,7.49,7.28,7.47,6.99,6.74,6.28
2,covid,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.82,5.09,4.51
3,hiv,1.07,1.06,1.04,1.05,1.00,0.96,0.92,0.76,0.83,0.76,0.80,0.78,0.70,0.68,0.61,0.58
4,mouse,1.91,1.88,1.78,1.79,1.82,1.77,1.69,1.63,1.56,1.57,1.58,1.49,1.47,1.43,1.35,1.28


# models

## polynom

In [62]:
def make_polynom(df_model, st=3, draw_plot=False, roky=(2007, 2022), test=2):
    word_polyn = {}
    for word in df_model['index']:
        x = list(map(float, df_model.columns[1:]))

        y =  df_model[df_model['index']==word]
        y = list(y.values[0][1:])

        koef = np.polyfit(x[:-test],y[:-test], st)
        p = np.poly1d(koef)

        if draw_plot:
            xp = list(range(roky[0], roky[1]+3))
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='reálne dáta'))
            fig.add_trace(go.Scatter(x=xp, y=p(xp),
                    mode='lines',
                    name='predpoveď'))
            fig.update_layout(title=f'{word.upper()} - polynóm {st}.stupňa',
                   xaxis_title='roky',
                   yaxis_title='percentuálne zastúpenie',
                   width=500,
                   height=500)
            fig.show()

        word_polyn[word] = p(xp[-2:])

        #print(word)
        #print(p(xp[-2:]))
        #print('r2 score:' , r2_score(y,p(x)))
    #return word_polyn, 

In [63]:
make_polynom(df_model, draw_plot=True)

# diff next mean

In [66]:
def make_diff(df_model):
    word_mean = {}
    i = 0
    for word in df_model['index']:
        rozdiel = df_model.transpose().iloc[2:,i] - df_model.transpose().shift(1).iloc[2:,i]
        if i==0:
            diff_next = pd.DataFrame({word:rozdiel})
        else:
            diff_next[word]=rozdiel        
        i += 1
    return diff_next

def make_mean(df_model, roll=2, priemer=False, draw_plot=False, roky=(2007, 2022), test=2):
    df_diff = make_diff(df_model)
    word_mean = {}
    i = 0
    for word in df_diff.columns:
        if priemer:
            rozdiel = df_diff.iloc[:,i].rolling(roll).mean().iloc[-roll-test:-test,].mean()
        else:
            rozdiel = df_diff.iloc[:,i].rolling(roll).mean().iloc[-test-1,]

        last = df_model.iloc[i,-test-1]
        last_rozdiel = [last+rozdiel*(i+1) for i in range(test+2)]
        word_mean[word] = last_rozdiel
        
        i += 1

        if draw_plot:
            x = list(range(roky[0], roky[1]+1))
            xp = []
            
            xp = [roky[1]-i for i in range(test, -3, -1)]
            y =  df_model[df_model['index']==word]
            y = list(y.values[0][1:])
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='reálne dáta'))
            fig.add_trace(go.Scatter(x=xp, y=[last]+last_rozdiel,
                    mode='lines',
                    name='predpoveď'))
            fig.update_layout(title=f'{word.upper()} - mean',
                   xaxis_title='roky',
                   yaxis_title='percentuálne zastúpenie',
                   width=500,
                   height=500)
            fig.show()
        #print('r2 score:' , r2_score(y[-test:], last_rozdiel[:-2]))
    
    #return word_mean

In [67]:
make_mean(df_model, draw_plot=True)

## ARIMA

In [68]:
def make_arima(df_model, test=2, roky=(2007, 2022)):
    word_arima = {}
    for word in df_model['index']:
        x = list(map(float, df_model.columns[1:]))

        y =  df_model[df_model['index']==word]
        y = list(y.values[0][1:])
        xp = list(range(roky[0], roky[1]+3))

        model = ARIMA(y[:-test], order=(1,2, 1))

        res = model.fit()
        vysl = res.predict(start=2, end=2026-2007)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y[1:],
                mode='lines',
                name='reálne dáta'))
        fig.add_trace(go.Scatter(x=xp, y=vysl,
                mode='lines',
                name='predpoveď'))
        fig.update_layout(title=f'{word.upper()} - ARIMA',
                xaxis_title='roky',
                yaxis_title='percentuálne zastúpenie',
                width=500,
                height=500)
        fig.show()

        word_arima[word] = vysl

        #print(word)
        #print(list(vysl))

    #return word_arima

In [69]:
make_arima(df_model)